In [ ]:
from google.colab import drive
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn import model_selection
from nltk.tokenize import word_tokenize
import nltk
from sklearn.metrics import accuracy_score
nltk.download('punkt')
import codecs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.105.239.170:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 3190145941591977215),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, 6583136922467624270),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 797745322997884523),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8776589540353923524),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, -2361822895174045667),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 5378676407677017773),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, -3486826876301646085),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, -746593129861577563),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.105.239.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.105.239.170:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2021-06-16 03:33:51--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2021-06-16 03:33:51--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2021-06-16 03:33:51--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [ ]:
!unzip glove.twitter.27B.zip
# !unzip wiki-news-300d-1M.vec.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [ ]:
# class GloveModels:
#   def __init__(self,dim,path):
#       df = pd.read_csv(path)
#       # self.encoder = LabelEncoder()
#       self.models_glove = {}
#       self.history_glove = {}
      
#       self.models_fasttext = {}
#       self.history_fasttext = {}
#       # self.accuracy_glove ={}
#       df['text'] = df['text'].apply(lambda x: str(x))
#       self.dfs_train = df[df['kfold']!=0]
#       self.dfs_valid = df[df['kfold']==0]
#       self.read_glove(dim)
#       self.data_per_fold_glove(dim)
#       # self.read_fasttext()
#       # self.data_per_fold_fasttext()

#   def read_glove(self):
#       embeddings_index = {}
#       f = open('glove.twitter.27B.'+str(dim)+'d.txt')
#       for line in f:
#           values = line.split()
#           word = values[0]
#           coefs = np.asarray(values[1:], dtype='float32')
#           embeddings_index[word] = coefs
#       f.close()

#       print('Found %s word vectors.' % len(embeddings_index))
#       self.glove_embeddings_index = embeddings_index

#   # def read_fasttext(self):
#   #     embeddings_index = {}
#   #     f = codecs.open('wiki-news-300d-1M.vec', encoding='utf-8')
#   #     for line in f:
#   #         values = line.rstrip().rsplit(' ')
#   #         word = values[0]
#   #         coefs = np.asarray(values[1:], dtype='float32')
#   #         embeddings_index[word] = coefs
#   #     f.close()
#   #     print('found %s word vectors' % len(embeddings_index))
#   #     self.fasttext_embeddings_index = embeddings_index

#   def create_glove_embedding_matrix(self,embed_dim):
#       train = self.dfs_train
#       test = self.dfs_valid
#       x_train = train.text.values
#       y_train = train.lbl_num.values
#       x_test = test.text.values
#       y_test = test.lbl_num.values
#       word_tokenizer = Tokenizer()
#       word_tokenizer.fit_on_texts(x_train)
#       vocab_length = len(word_tokenizer.word_index) + 1
#       x_train_emb = word_tokenizer.texts_to_sequences(x_train)
#       x_test_emb = word_tokenizer.texts_to_sequences(x_test)
      
#       word_count = lambda sentence: len(word_tokenize(sentence))
#       longest_sentence = max(np.concatenate((x_train,x_test),axis = 0), key=word_count)
#       length_long_sentence = len(word_tokenize(longest_sentence))
#       # print('length_long_sentence: '+str(length_long_sentence))
#       self.glove_wt = word_tokenizer
#       self.lls = length_long_sentence
#       x_train_padded = pad_sequences(x_train_emb, length_long_sentence, padding='post',truncating = 'post')
#       x_test_padded = pad_sequences(x_test_emb, length_long_sentence, padding='post',truncating = 'post')

#       # encoded_Y_tr = self.encoder.fit_transform(y_train)
#       # encoded_Y_ts = self.encoder.fit_transform(y_test)
#       dummy_y_tr = np_utils.to_categorical(y_train)
#       dummy_y_ts = np_utils.to_categorical(y_test)

#       embedding_matrix = np.zeros((vocab_length, embed_dim))
#       for word, index in word_tokenizer.word_index.items():
#           embedding_vector = self.glove_embeddings_index.get(word)
#           if embedding_vector is not None:
#               embedding_matrix[index] = embedding_vector

#       # print(x_train_padded)
#       # print(x_test_padded)
#       return ((tf.convert_to_tensor(x_train_padded,np.int32),tf.convert_to_tensor(dummy_y_tr,np.float32)),(tf.convert_to_tensor(x_test_padded,np.int32),tf.convert_to_tensor(dummy_y_ts,np.float32)),(vocab_length,embedding_matrix,embed_dim,length_long_sentence))

#   # def create_fasttext_embedding_matrix(self,embed_dim):
#   #     train = self.dfs_train
#   #     test = self.dfs_valid
#   #     x_train = train.text.values
#   #     y_train = train.lbl_num.values
#   #     x_test = test.text.values
#   #     y_test = test.lbl_num.values
#   #     word_tokenizer = Tokenizer(num_words = 50000,oov_token="<OOV>")
#   #     word_tokenizer.fit_on_texts(x_train)
#   #     vocab_length = len(word_tokenizer.word_index) + 1
#   #     x_train_emb = word_tokenizer.texts_to_sequences(x_train)
#   #     x_test_emb = word_tokenizer.texts_to_sequences(x_test)
      
#   #     word_count = lambda sentence: len(word_tokenize(sentence))
#   #     longest_sentence = max(np.concatenate((x_train,x_test),axis = 0), key=word_count)
#   #     length_long_sentence = len(word_tokenize(longest_sentence))

#   #     self.fasttext_wt = word_tokenizer
#   #     self.lls = length_long_sentence
#   #     x_train_padded = pad_sequences(x_train_emb, length_long_sentence, padding='post',truncating = 'post')
#   #     x_test_padded = pad_sequences(x_test_emb, length_long_sentence, padding='post',truncating = 'post')

#   #     # encoded_Y_tr = self.encoder.fit_transform(y_train)
#   #     # encoded_Y_ts = self.encoder.fit_transform(y_test)
#   #     dummy_y_tr = np_utils.to_categorical(y_train)
#   #     dummy_y_ts = np_utils.to_categorical(y_test)
#   #     # self.df_labels = pd.DataFrame.from_dict({'y_train':y_train,'y_test':y_test,'dummy_tr':dummy_y_tr,'dummy_y_ts':dummy_y_ts})

#   #     embedding_matrix = np.zeros((vocab_length, embed_dim))
#   #     for word, index in word_tokenizer.word_index.items():
#   #         embedding_vector = self.fasttext_embeddings_index.get(word)
#   #         if embedding_vector is not None:
#   #             embedding_matrix[index] = embedding_vector

#   #     # print(x_train_padded)
#   #     # print(x_test_padded)
#   #     return ((tf.convert_to_tensor(x_train_padded,np.int32),tf.convert_to_tensor(dummy_y_tr,np.float32)),(tf.convert_to_tensor(x_test_padded,np.int32),tf.convert_to_tensor(dummy_y_ts,np.float32)),(vocab_length,embedding_matrix,embed_dim,length_long_sentence))


#   def get_glove_preprocessing_params(self):
#       return self.glove_wt,self.lls

#   # def get_fasttext_preprocessing_params(self):
#   #     return self.fasttext_wt,self.lls

#   # def get_labels_df(self):
#   #     return self.df_labels
  
#   def data_per_fold_glove(self,dim):
#       # self.data_glove = {}
#       embed_dim = dim
#       self.data_glove = self.create_glove_embedding_matrix(embed_dim)

#   # def data_per_fold_fasttext(self,dim):
#   #     # self.data_fasttext = {}
#   #     embed_dim = dim
#   #     self.data_fasttext = self.create_fasttext_embedding_matrix(embed_dim)


#   def process_glove(self):
#       models = [self.CONV1D_BiLSTM]#self.SEP_CONV1D_BiRNN,self.SEP_CONV1D_BiGRU,self.BiGRU,self.BiLSTM,self.BiRNN]
#       for mod in models:
#           name = str(mod)
#           name = name.replace('self.','')
#           m,h = self.run_training(mod)
#           self.models_glove[name] = m
#           self.history_glove[name] = h
#           # self.accuracy_glove['Test accuracy score for '+name] = acc

#   # def process_fasttext(self):
#   #     models = [self.CONV1D]
#   #     for mod in models:
#   #         name = str(mod)
#   #         name = name.replace('self.','')
#   #         m,h = self.run_training(mod)
#   #         self.models_fasttext[name] = m
#   #         self.history_fasttext[name] = h
#   #         # self.accuracy_glove['Test accuracy score for '+name] = acc
    
#   def get_models_glove(self):
#     return self.models_glove

#   def get_history_glove(self):
#     return self.history_glove

#   # def get_models_fasttext(self):
#   #   return self.models_fasttext

#   # def get_history_fasttext(self):
#   #   return self.history_fasttext


#   def run_training(self,model):
#       mod = []
#       his = []
#       x,h= model()
#       mod.append(x)
#       his.append(h)
    
      
#       return mod,his

#   # def SEP_CONV1D(self):
#   #     data = self.data_fasttext
#   #     x_tr,y_tr = data[0]
#   #     x_ts,y_ts = data[1]
#   #     vl,mat,dim,len_sen = data[2]
      
#   #     with strategy.scope():
#   #         model = tf.keras.models.Sequential([
#   #             tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_embedding"),
#   #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_embedding"),
#   #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_embedding"),
#   #             tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#   #             tf.keras.layers.MaxPool1D(),
#   #             tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#   #             tf.keras.layers.MaxPool1D(),
#   #             tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#   #             tf.keras.layers.MaxPool1D(),
#   #             tf.keras.layers.Dropout(0.5),
#   #             tf.keras.layers.Flatten(),
#   #             tf.keras.layers.Dense(512, activation='sigmoid'),
#   #             tf.keras.layers.Dense(256, activation='sigmoid'),
#   #             tf.keras.layers.Dense(128, activation='sigmoid'),
#   #             tf.keras.layers.Dense(64, activation='sigmoid'),
#   #             tf.keras.layers.Dense(5, activation='softmax')])

#   #         model.compile(loss='categorical_crossentropy',
#   #         optimizer=tf.keras.optimizers.Adam(),
#   #         metrics=['accuracy'])
#   #     x = model.to_json()
#   #     print("\n SEPERABLE CONV1D \n")
#   #     model.summary()
#   #     his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#   #     # y_pred = model.predict(x_ts)
#   #     return x,his





#   def CONV1D(self):
#       data = self.data_glove
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_embedding"),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(5, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n CONV1D \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return model,his#,accuracy_score(y_ts,y_pred)

#   def BiLSTM(self):
#       data = self.data_glove
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiLSTM_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiLSTM_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiLSTM_embedding"),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               # tf.keras.layers.Dense(5, activation='softmax')
#               tf.keras.layers.Dense(2, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n BiLSTM \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return model,his#,accuracy_score(y_ts,y_pred)

#   def SEP_CONV1D_BiLSTM(self):
#       data = self.data_fasttext
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiLSTM_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiLSTM_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_BiLSTM_embedding"),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(5, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n SEPARABLE CONV1D BiLSTM \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return x,his #,accuracy_score(y_ts,y_pred)


#   def CONV1D_BiLSTM(self):
#       data = self.data_glove
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiLSTM_embedding"),
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiLSTM_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiLSTM_embedding"),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(5, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n CONV1D BiLSTM \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return model,his #,accuracy_score(y_ts,y_pred)


#   def BiRNN(self):
#       data = self.data_fasttext
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiRNN_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiRNN_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiRNN_embedding"),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(5, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n BiRNN \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return x,his #,accuracy_score(y_ts,y_pred)

#   def SEP_CONV1D_BiRNN(self):
#       data = self.data_fasttext
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiRNN_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiRNN_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_BiRNN_embedding"),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(5, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n SEPARABLE CONV1D BiRNN \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return model,his #,accuracy_score(y_ts,y_pred)

  
#   def CONV1D_BiRNN(self):
#       data = self.data_fasttext
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiRNN_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiRNN_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiRNN_embedding"),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(5, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n CONV1D BiRNN \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return x,his #,accuracy_score(y_ts,y_pred)

#   def BiGRU(self):
#       data = self.data_fasttext
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiGRU_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiGRU_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiGRU_embedding"),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               # tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(5, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n BiGRU \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return x,his #,accuracy_score(y_ts,y_pred)
      
#   def CONV1D_BiGRU(self):
#       data = self.data_glove
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiGRU_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiGRU_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiGRU_embedding"),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.Conv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(2, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n CONV1D BiGRU \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return model,his #,accuracy_score(y_ts,y_pred)
      
#   def SEP_CONV1D_BiGRU(self):
#       data = self.data_fasttext
#       x_tr,y_tr = data[0]
#       x_ts,y_ts = data[1]
#       vl,mat,dim,len_sen = data[2]
      
#       with strategy.scope():
#           model = tf.keras.models.Sequential([
#               tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiGRU_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiGRU_embedding"),
#               # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_BiGRU_embedding"),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
#               tf.keras.layers.MaxPool1D(),
#               # tf.keras.layers.Dropout(0.5),
#               tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
#               tf.keras.layers.Flatten(),
#               tf.keras.layers.Dense(512, activation='sigmoid'),
#               tf.keras.layers.Dense(256, activation='sigmoid'),
#               tf.keras.layers.Dense(128, activation='sigmoid'),
#               tf.keras.layers.Dense(64, activation='sigmoid'),
#               tf.keras.layers.Dense(5, activation='softmax')])

#           model.compile(loss='categorical_crossentropy',
#           optimizer=tf.keras.optimizers.Adam(),
#           metrics=['accuracy'])
#       x = model.to_json()
#       print("\n SEPARABLE CONV1D BiGRU \n")
#       model.summary()
#       his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
#       # y_pred = model.predict(x_ts)
#       return x,his #,accuracy_score(y_ts,y_pred)

In [ ]:
class GloveModels:
  def __init__(self,path):
    df = pd.read_csv(path)
    df['text'] = df['text'].apply(lambda x: str(x))
    self.dfs_train = df[df['kfold']!=0]
    self.dfs_valid = df[df['kfold']==0]



  def process(self):
    self.read_glove()
    self.data_per_fold_glove()
    return self.glove_wt,self.lls
    

  def read_glove(self):
      embeddings_index = {}
      f = open('glove.twitter.27B.200d.txt')
      for line in f:
          values = line.split()
          word = values[0]
          coefs = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = coefs
      f.close()

      print('Found %s word vectors.' % len(embeddings_index))
      self.glove_embeddings_index = embeddings_index
  
  def data_per_fold_glove(self):
      # self.data_glove = {}
      embed_dim = 200
      self.data_glove = self.create_glove_embedding_matrix(embed_dim)

  def create_glove_embedding_matrix(self,embed_dim):
    train = self.dfs_train
    test = self.dfs_valid
    x_train = train.text.values
    y_train = train.lbl_num.values
    x_test = test.text.values
    y_test = test.lbl_num.values
    word_tokenizer = Tokenizer()
    word_tokenizer.fit_on_texts(x_train)
    vocab_length = len(word_tokenizer.word_index) + 1
    x_train_emb = word_tokenizer.texts_to_sequences(x_train)
    x_test_emb = word_tokenizer.texts_to_sequences(x_test)
    
    word_count = lambda sentence: len(word_tokenize(sentence))
    longest_sentence = max(np.concatenate((x_train,x_test),axis = 0), key=word_count)
    length_long_sentence = len(word_tokenize(longest_sentence))
    # print('length_long_sentence: '+str(length_long_sentence))
    self.glove_wt = word_tokenizer
    self.lls = length_long_sentence
    x_train_padded = pad_sequences(x_train_emb, length_long_sentence, padding='post',truncating = 'post')
    x_test_padded = pad_sequences(x_test_emb, length_long_sentence, padding='post',truncating = 'post')

    # encoded_Y_tr = self.encoder.fit_transform(y_train)
    # encoded_Y_ts = self.encoder.fit_transform(y_test)
    dummy_y_tr = np_utils.to_categorical(y_train)
    dummy_y_ts = np_utils.to_categorical(y_test)

    embedding_matrix = np.zeros((vocab_length, embed_dim))
    for word, index in word_tokenizer.word_index.items():
        embedding_vector = self.glove_embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

    # print(x_train_padded)
    # print(x_test_padded)
    return ((tf.convert_to_tensor(x_train_padded,np.int32),tf.convert_to_tensor(dummy_y_tr,np.float32)),(tf.convert_to_tensor(x_test_padded,np.int32),tf.convert_to_tensor(dummy_y_ts,np.float32)),(vocab_length,embedding_matrix,embed_dim,length_long_sentence))

  # def CONV1D(self,v):
  #     data = self.data_glove
  #     x_tr,y_tr = data[0]
  #     x_ts,y_ts = data[1]
  #     vl,mat,dim,len_sen = data[2]
      
      
  #     with strategy.scope():
  #         model = tf.keras.models.Sequential([
  #             tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200D_CONV1D_embedding"),
  #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_embedding"),
  #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_embedding"),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             tf.keras.layers.Dropout(0.5),
  #             tf.keras.layers.Flatten(),
  #             tf.keras.layers.Dense(512, activation='sigmoid'),
  #             tf.keras.layers.Dense(256, activation='sigmoid'),
  #             tf.keras.layers.Dense(128, activation='sigmoid'),
  #             tf.keras.layers.Dense(64, activation='sigmoid'),
  #             tf.keras.layers.Dense(v, activation='softmax')])

  #         model.compile(loss='categorical_crossentropy',
  #         optimizer=tf.keras.optimizers.Adam(),
  #         metrics=['accuracy'])
  #     # x = model.to_json()
  #     print("\n CONV1D \n")
  #     model.summary()
  #     his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=10),ModelCheckpoint(filepath='/content/drive/MyDrive/GLOVE_100/sarcasm/sarcasm_CONV1D.h5', monitor='val_loss', save_best_only=True)])
  #     # y_pred = model.predict(x_ts)
  #     return model,his#,accuracy_score(y_ts,y_pred)



  def CONV1D_BiLSTM(self,v):
      data = self.data_glove
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiLSTM_embedding"),
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiLSTM_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiLSTM_embedding"),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(v, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      # x = model.to_json()
      print("\n CONV1D BiLSTM \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=10),ModelCheckpoint(filepath='/content/drive/MyDrive/GLOVE_200/sarcasm/sarcasm_CONV1D_BiLSTM.h5', monitor='val_loss', save_best_only=True)])
      # y_pred = model.predict(x_ts)
      return model,his #,accuracy_score(y_ts,y_pred)


    
  def BiRNN(self,v):
      data = self.data_glove
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200D_BiRNN_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiRNN_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiRNN_embedding"),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(v, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      # x = model.to_json()
      print("\n BiRNN \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=10),ModelCheckpoint(filepath='/content/drive/MyDrive/GLOVE_200/sarcasm/sarcasm_BiRNN.h5', monitor='val_loss', save_best_only=True)])
      # y_pred = model.predict(x_ts)
      return model,his #,accuracy_score(y_ts,y_pred)

  
  # def CONV1D_BiRNN(self,v):
  #     data = self.data_glove
  #     x_tr,y_tr = data[0]
  #     x_ts,y_ts = data[1]
  #     vl,mat,dim,len_sen = data[2]
      
      
  #     with strategy.scope():
  #         model = tf.keras.models.Sequential([
  #             tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200D_CONV1D_BiRNN_embedding"),
  #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiRNN_embedding"),
  #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiRNN_embedding"),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             # tf.keras.layers.Dropout(0.5),
  #             tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
  #             tf.keras.layers.Flatten(),
  #             tf.keras.layers.Dense(512, activation='sigmoid'),
  #             tf.keras.layers.Dense(256, activation='sigmoid'),
  #             tf.keras.layers.Dense(128, activation='sigmoid'),
  #             tf.keras.layers.Dense(64, activation='sigmoid'),
  #             tf.keras.layers.Dense(v, activation='softmax')])

  #         model.compile(loss='categorical_crossentropy',
  #         optimizer=tf.keras.optimizers.Adam(),
  #         metrics=['accuracy'])
  #     # x = model.to_json()
  #     print("\n CONV1D BiRNN \n")
  #     model.summary()
  #     his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=10),ModelCheckpoint(filepath='/content/drive/MyDrive/GLOVE_100/sarcasm/sarcasm_CONV1D_BiRNN.h5', monitor='val_loss', save_best_only=True)])
  #     # y_pred = model.predict(x_ts)
  #     return model,his #,accuracy_score(y_ts,y_pred)

  # def BiGRU(self,v):
  #     data = self.data_glove
  #     x_tr,y_tr = data[0]
  #     x_ts,y_ts = data[1]
  #     vl,mat,dim,len_sen = data[2]
      
      
  #     with strategy.scope():
  #         model = tf.keras.models.Sequential([
  #             tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200D_BiGRU_embedding"),
  #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiGRU_embedding"),
  #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiGRU_embedding"),
  #             # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
  #             # tf.keras.layers.MaxPool1D(),
  #             # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
  #             # tf.keras.layers.MaxPool1D(),
  #             # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
  #             # tf.keras.layers.MaxPool1D(),
  #             # tf.keras.layers.Dropout(0.5),
  #             tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
  #             tf.keras.layers.Flatten(),
  #             tf.keras.layers.Dense(512, activation='sigmoid'),
  #             tf.keras.layers.Dense(256, activation='sigmoid'),
  #             tf.keras.layers.Dense(128, activation='sigmoid'),
  #             tf.keras.layers.Dense(64, activation='sigmoid'),
  #             tf.keras.layers.Dense(v, activation='softmax')])

  #         model.compile(loss='categorical_crossentropy',
  #         optimizer=tf.keras.optimizers.Adam(),
  #         metrics=['accuracy'])
  #     # x = model.to_json()
  #     print("\n BiGRU \n")
  #     model.summary()
  #     his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=10),ModelCheckpoint(filepath='/content/drive/MyDrive/GLOVE_100/sarcasm/sarcasm_BiGRU.h5', monitor='val_loss', save_best_only=True)])
  #     # y_pred = model.predict(x_ts)
  #     return model,his #,accuracy_score(y_ts,y_pred)
      
  # def CONV1D_BiGRU(self,v):
  #     data = self.data_glove
  #     x_tr,y_tr = data[0]
  #     x_ts,y_ts = data[1]
  #     vl,mat,dim,len_sen = data[2]
      
      
  #     with strategy.scope():
  #         model = tf.keras.models.Sequential([
  #             tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200D_CONV1D_BiGRU_embedding"),
  #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiGRU_embedding"),
  #             # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiGRU_embedding"),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             tf.keras.layers.Conv1D(32,3,activation='relu'),
  #             tf.keras.layers.MaxPool1D(),
  #             # tf.keras.layers.Dropout(0.5),
  #             tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
  #             tf.keras.layers.Flatten(),
  #             tf.keras.layers.Dense(512, activation='sigmoid'),
  #             tf.keras.layers.Dense(256, activation='sigmoid'),
  #             tf.keras.layers.Dense(128, activation='sigmoid'),
  #             tf.keras.layers.Dense(64, activation='sigmoid'),
  #             tf.keras.layers.Dense(v, activation='softmax')])

  #         model.compile(loss='categorical_crossentropy',
  #         optimizer=tf.keras.optimizers.Adam(),
  #         metrics=['accuracy'])
  #     # x = model.to_json()
  #     print("\n CONV1D BiGRU \n")
  #     model.summary()
  #     his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=10),ModelCheckpoint(filepath='/content/drive/MyDrive/GLOVE_100/sarcasm/sarcasm_CONV1D_BiGRU.h5', monitor='val_loss', save_best_only=True)])
  #     # y_pred = model.predict(x_ts)
  #     return model,his #,accuracy_score(y_ts,y_pred)

  def BiLSTM(self,v):
    data = self.data_glove
    x_tr,y_tr = data[0]
    x_ts,y_ts = data[1]
    vl,mat,dim,len_sen = data[2]
    
    with strategy.scope():
        model = tf.keras.models.Sequential([
            tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiLSTM_embedding"),
            # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiLSTM_embedding"),
            # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiLSTM_embedding"),
            # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
            # tf.keras.layers.MaxPool1D(),
            # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
            # tf.keras.layers.MaxPool1D(),
            # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
            # tf.keras.layers.MaxPool1D(),
            # tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='sigmoid'),
            tf.keras.layers.Dense(256, activation='sigmoid'),
            tf.keras.layers.Dense(128, activation='sigmoid'),
            tf.keras.layers.Dense(64, activation='sigmoid'),
            # tf.keras.layers.Dense(5, activation='softmax')
            tf.keras.layers.Dense(v, activation='softmax')])

        model.compile(loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy'])

    print("\n BiLSTM \n")
    model.summary()
    his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=10),ModelCheckpoint(filepath='/content/drive/MyDrive/GLOVE_200/sarcasm/sarcasm_BiLSTM.h5', monitor='val_loss', save_best_only=True)])
    # y_pred = model.predict(x_ts)
    return model,his#,accuracy_score(y_ts,y_pred)

  def SEP_CONV1D_BiGRU(self,v):
      data = self.data_glove
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
    
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_BiGRU_embedding"),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(v, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      # x = model.to_json()
      print("\n SEPARABLE CONV1D BiGRU \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=10),ModelCheckpoint(filepath='/content/drive/MyDrive/GLOVE_200/sarcasm/sarcasm_SEP_CONV1D_BiGRU.h5', monitor='val_loss', save_best_only=True)])
      # y_pred = model.predict(x_ts)
      return model,his #,accuracy_score(y_ts,y_pred)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/train_sarc_final_with_folds.csv')

In [ ]:
# models = ['CONV1D','CONV1D_BiRNN','BiRNN','BiGRU','CONV1D_BiGRU']
sarc_models = {}
sarc_models_clf = GloveModels('/content/drive/MyDrive/datasets/train_sarc_final_with_folds.csv')
sarc_param_glove = {}
sarc_param_glove['Tokenizer'],sarc_param_glove['long_sen_num'] = sarc_models_clf.process()
# sarc_models['CONV1D_BiLSTM'] = sarc_models_clf.CONV1D_BiLSTM(2)
# # sarc_models['CONV1D_BiRNN'] = sarc_models_clf.CONV1D_BiRNN(2)
# # sarc_models['CONV1D_BiGRU'] = sarc_models_clf.CONV1D_BiGRU(2)
# sarc_models['BiLSTM'] = sarc_models_clf.BiLSTM(2)
# sarc_models['BiRNN'] = sarc_models_clf.BiRNN(2)
sarc_models['SEP_CONV1D_BiGRU'] = sarc_models_clf.SEP_CONV1D_BiGRU(2)


Found 1193514 word vectors.

 SEPARABLE CONV1D BiGRU 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GL_200_SEP_CONV1D_BiGRU_embe (None, 163, 200)          16891400  
_________________________________________________________________
separable_conv1d (SeparableC (None, 161, 32)           7032      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 80, 32)            0         
_________________________________________________________________
separable_conv1d_1 (Separabl (None, 78, 32)            1152      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 39, 32)            0         
_________________________________________________________________
separable_conv1d_2 (Separabl (None, 37, 32)            1152      
_________________________________________________________________

In [ ]:
import pickle
with open('/content/drive/MyDrive/GLOVE_200/sarcasm/sarcasm_params.pickle','wb') as f:
  pickle.dump(sarc_param_glove,f)

In [ ]:
sarc_models['CONV1D'][0]

In [ ]:
x

{'Tokenizer': <keras_preprocessing.text.Tokenizer at 0x7f93a0b73090>,
 'long_sen_num': 163}

In [ ]:
sent_models = {}
sent_models_clf = GloveModels('/content/drive/MyDrive/datasets/tr_sen_fin.csv')
sent_param_glove = {}
sent_param_glove['Tokenizer'],sent_param_glove['long_sen_num'] = sent_models_clf.process()
sent_models['CONV1D'] = sent_models_clf.CONV1D(5)
sent_models['CONV1D_BiRNN'] = sent_models_clf.CONV1D_BiRNN(5)
sent_models['CONV1D_BiGRU'] = sent_models_clf.CONV1D_BiGRU(5)
sent_models['BiGRU'] = sent_models_clf.BiGRU(5)
sent_models['BiRNN'] = sent_models_clf.BiRNN(5)
import pickle
with open('/content/drive/MyDrive/GLOVE_100/sentiment/sentiment_params.pickle','wb') as f:
  pickle.dump(sent_param_glove,f)
for _,x in enumerate(sent_models.keys()):
  sent_models[x][0].save('/content/drive/MyDrive/GLOVE_100/sentiment/sentiment_'+x+'.h5')

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
